In [161]:
import pandas as pd
import numpy as np

In [162]:
data_types = {
    "Store": "int16",
    "DayOfWeek": "int8",
    "CompetitionDistance": "float32",
    "CompetitionOpenSinceMonth": "float32",
    "CompetitionOpenSinceYear": "float32",
    "CompetitionDistanceMissing": "bool",
    "CompetitionOpenMissing": "bool",
    "StateHoliday": "category",
    "SchoolHoliday": "int8",
    "Promo": "bool",
    "Promo2": "bool",
    "Promo2SinceYear": "float32",
    "Promo2SinceWeek": "float32",
    "PromoInterval": "category",
    "StoreType": "category",
    "Assortment": "category",
    "Sales": "float32"
}

In [163]:
data = pd.read_csv(
    "../Data/Preprocessed_data/Cleaned Data/cleaned_training_data.csv",
    dtype = data_types,
    parse_dates=["Date"],
    index_col="Date_index",
    encoding="utf-8",
)

In [164]:
data.head()

,Store,Date,DayOfWeek,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,CompetitionDistanceMissing,CompetitionOpenMissing,StateHoliday,SchoolHoliday,Promo,Promo2,Promo2SinceYear,Promo2SinceWeek,PromoInterval,StoreType,Assortment,Sales
Date_index,,,,,,,,,,,,,,,,,,
2015-07-31,1,2015-07-31,5,1270.0,9.0,2008.0,False,False,0,1,True,False,0.0,0.0,no_promo,c,a,8.568457
2015-07-31,2,2015-07-31,5,570.0,11.0,2007.0,False,False,0,1,True,True,2010.0,13.0,"Jan,Apr,Jul,Oct",a,a,8.710125
2015-07-31,3,2015-07-31,5,14130.0,12.0,2006.0,False,False,0,1,True,True,2011.0,14.0,"Jan,Apr,Jul,Oct",a,a,9.025696
2015-07-31,4,2015-07-31,5,620.0,9.0,2009.0,False,False,0,1,True,False,0.0,0.0,no_promo,c,c,9.546455
2015-07-31,5,2015-07-31,5,29910.0,4.0,2015.0,False,False,0,1,True,False,0.0,0.0,no_promo,a,a,8.480944


In [165]:
data.PromoInterval.isna().sum()

np.int64(0)

In [166]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 844338 entries, 2015-07-31 to 2013-01-01
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Store                       844338 non-null  int16         
 1   Date                        844338 non-null  datetime64[ns]
 2   DayOfWeek                   844338 non-null  int8          
 3   CompetitionDistance         842152 non-null  float32       
 4   CompetitionOpenSinceMonth   844338 non-null  float32       
 5   CompetitionOpenSinceYear    844338 non-null  float32       
 6   CompetitionDistanceMissing  844338 non-null  bool          
 7   CompetitionOpenMissing      844338 non-null  bool          
 8   StateHoliday                844338 non-null  category      
 9   SchoolHoliday               844338 non-null  int8          
 10  Promo                       844338 non-null  bool          
 11  Promo2                      844

## Working on Date column
I will extract year month and day cuz most models can't understand Timestamp as it is

In [167]:
def TransformDateFeature(data):
    """Extracts date features from the 'Date' column in the DataFrame."""
    data["Year"] = data["Date"].dt.year
    data["Month"] = data["Date"].dt.month
    data["Day"] = data["Date"].dt.day

    data["DayOfYear"] = data["Date"].dt.dayofyear
    data["WeekOfYear"] = data["Date"].dt.isocalendar().week.astype(int)

    # Sales often spike at the beginning and end of the month.
    # This flag captures the last few days of the month.
    data["IsLastDayOfMonth"] = data["Date"].dt.is_month_end.astype(bool)
    import numpy as np

    # 1. Cyclical Encoding for DayOfWeek -- it's important to capture the cyclical nature of days in a week
    # NN may misinterpret Monday (1) and Sunday (7) as being far apart numerically the same as 1-12 for months
    data["DayOfWeek_sin"] = np.sin(2 * np.pi * data["DayOfWeek"] / 7).astype(np.float32)
    data["DayOfWeek_cos"] = np.cos(2 * np.pi * data["DayOfWeek"] / 7).astype(np.float32)

    data["Month_sin"] = np.sin(2 * np.pi * data["Month"] / 12).astype(np.float32)
    data["Month_cos"] = np.cos(2 * np.pi * data["Month"] / 12).astype(np.float32)

    data["IsWeekend"] = data["DayOfWeek"].isin([6, 7]).astype(bool)
    data["IsMonthEnd"] = data["Date"].dt.is_month_end.astype(bool)
    data["IsMonthStart"] = data["Date"].dt.is_month_start.astype(bool)

    data.drop("Date", axis=1, inplace=True)
    return data

In [168]:
data = TransformDateFeature(data)
data.head()

,Store,DayOfWeek,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,CompetitionDistanceMissing,CompetitionOpenMissing,StateHoliday,SchoolHoliday,Promo,Promo2,Promo2SinceYear,Promo2SinceWeek,PromoInterval,StoreType,Assortment,Sales,Year,Month,Day,DayOfYear,WeekOfYear,IsLastDayOfMonth,DayOfWeek_sin,DayOfWeek_cos,Month_sin,Month_cos,IsWeekend,IsMonthEnd,IsMonthStart
Date_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-07-31,1,5,1270.0,9.0,2008.0,False,False,0,1,True,False,0.0,0.0,no_promo,c,a,8.568457,2015,7,31,212,31,True,-0.974928,-0.222521,-0.5,-0.866025,False,True,False
2015-07-31,2,5,570.0,11.0,2007.0,False,False,0,1,True,True,2010.0,13.0,"Jan,Apr,Jul,Oct",a,a,8.710125,2015,7,31,212,31,True,-0.974928,-0.222521,-0.5,-0.866025,False,True,False
2015-07-31,3,5,14130.0,12.0,2006.0,False,False,0,1,True,True,2011.0,14.0,"Jan,Apr,Jul,Oct",a,a,9.025696,2015,7,31,212,31,True,-0.974928,-0.222521,-0.5,-0.866025,False,True,False
2015-07-31,4,5,620.0,9.0,2009.0,False,False,0,1,True,False,0.0,0.0,no_promo,c,c,9.546455,2015,7,31,212,31,True,-0.974928,-0.222521,-0.5,-0.866025,False,True,False
2015-07-31,5,5,29910.0,4.0,2015.0,False,False,0,1,True,False,0.0,0.0,no_promo,a,a,8.480944,2015,7,31,212,31,True,-0.974928,-0.222521,-0.5,-0.866025,False,True,False


In [169]:
pd.set_option('display.max_columns', None)

In [170]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 844338 entries, 2015-07-31 to 2013-01-01
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   Store                       844338 non-null  int16   
 1   DayOfWeek                   844338 non-null  int8    
 2   CompetitionDistance         842152 non-null  float32 
 3   CompetitionOpenSinceMonth   844338 non-null  float32 
 4   CompetitionOpenSinceYear    844338 non-null  float32 
 5   CompetitionDistanceMissing  844338 non-null  bool    
 6   CompetitionOpenMissing      844338 non-null  bool    
 7   StateHoliday                844338 non-null  category
 8   SchoolHoliday               844338 non-null  int8    
 9   Promo                       844338 non-null  bool    
 10  Promo2                      844338 non-null  bool    
 11  Promo2SinceYear             844338 non-null  float32 
 12  Promo2SinceWeek             844338 non-null  float

#### Work on competition columns